In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import cv2
from datetime import datetime, timedelta

In [ ]:
df_ = pd.read_csv('/content/drive/MyDrive/Funnel_Analysis/user_data.csv')
df_btf = df_.copy()

In [ ]:
df_btf.head()

,user_id,stage,conversion
0,user_0,homepage,True
1,user_1,homepage,True
2,user_2,homepage,True
3,user_3,homepage,True
4,user_4,homepage,True


In [ ]:
df_t = df_btf[df_btf['conversion']==True]
df_t.head()

,user_id,stage,conversion
0,user_0,homepage,True
1,user_1,homepage,True
2,user_2,homepage,True
3,user_3,homepage,True
4,user_4,homepage,True


In [ ]:
df_t.shape

(13014, 3)

In [ ]:
df1 = df_t[df_t['stage']!='cart']

In [ ]:
df1['stage'].value_counts()

homepage        10000
product_page     2515
checkout           36
purchase           14
Name: stage, dtype: int64

In [ ]:
df = df1[df1['stage']!='checkout']

In [ ]:
df.shape

(12529, 3)

In [ ]:
df['stage'].value_counts()

homepage        10000
product_page     2515
purchase           14
Name: stage, dtype: int64

In [ ]:
df['conversion'].value_counts()

True    12529
Name: conversion, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12529 entries, 0 to 17172
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     12529 non-null  object
 1   stage       12529 non-null  object
 2   conversion  12529 non-null  bool  
dtypes: bool(1), object(2)
memory usage: 305.9+ KB


Untill now, I have considered only the true values and kept only 3 dimensions in stage: home page, product_page and purchase

# Adding random dates to the dataset

In [ ]:
start_date = datetime(2023, 11, 1)
end_date = datetime(2023, 11, 30)

In [ ]:
df['transaction_date'] = [start_date + timedelta(days=random.randint(0, (end_date - start_date).days)) for _ in range(len(df))]

<ipython-input-17-f22b5facbbaa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['transaction_date'] = [start_date + timedelta(days=random.randint(0, (end_date - start_date).days)) for _ in range(len(df))]


In [ ]:
df.head()

,user_id,stage,conversion,transaction_date
0,user_0,homepage,True,2023-11-06
1,user_1,homepage,True,2023-11-09
2,user_2,homepage,True,2023-11-23
3,user_3,homepage,True,2023-11-02
4,user_4,homepage,True,2023-11-29


Adding_values

In [ ]:
df[df['stage']=='purchase']['transaction_date'].nunique()

12

In [ ]:
df[df['stage']=='product_page']['transaction_date'].nunique()

30

In [ ]:
df[df['stage']=='homepage']['transaction_date'].nunique()

30

Since, purchase does not include all the dates, adding them, so that we can conduct the analysis properly

In [ ]:
all_dates = pd.date_range(start='2023-11-01', end='2023-11-30')
missing_dates = set(all_dates) - set(df[df['stage'] == 'purchase']['transaction_date'])

data_to_add = {
    'user_id': [f'user_{i}' for i in range(15000, 15000 + len(missing_dates))],
    'stage': ['purchase'] * len(missing_dates),
    'conversion': [True] * len(missing_dates),
    'transaction_date': list(missing_dates)
}

new_data_to_add = pd.DataFrame(data_to_add)

# Concatenate the new data with the existing DataFrame
df = pd.concat([df, new_data_to_add], ignore_index=True)

In [ ]:
df[df['stage']=='purchase']['transaction_date'].nunique()

30

The extra dates are successfully added

Adding a week column which has the week number of each week

In [ ]:
df['week'] = df['transaction_date'].dt.strftime('%Y-w%U')

In [ ]:
df.head()

,user_id,stage,conversion,transaction_date,week
0,user_0,homepage,True,2023-11-06,2023-w45
1,user_1,homepage,True,2023-11-09,2023-w45
2,user_2,homepage,True,2023-11-23,2023-w47
3,user_3,homepage,True,2023-11-02,2023-w44
4,user_4,homepage,True,2023-11-29,2023-w48


In [ ]:
week_label = df['week'].unique().tolist()

In [ ]:
week_label = sorted(week_label)

In [ ]:
week_label

['2023-w44', '2023-w45', '2023-w46', '2023-w47', '2023-w48']

In [ ]:
week1 = []
home_to_product_list = []
product_to_purchase_list = []
home_to_purchase_list = []

for i in week_label:
  week1.append(i)

  temp_df = df[df['week']==i]
  home_to_product= temp_df['stage'].value_counts()['product_page']/temp_df['stage'].value_counts()['homepage']
  product_to_purchase = temp_df['stage'].value_counts()['purchase']/temp_df['stage'].value_counts()['product_page']
  home_to_purchase = temp_df['stage'].value_counts()['purchase']/temp_df['stage'].value_counts()['homepage']
  home_to_product_list.append(home_to_product)
  product_to_purchase_list.append(product_to_purchase)
  home_to_purchase_list.append(home_to_purchase)

In [ ]:
len(week1) == len(home_to_product_list) == len(product_to_purchase_list) == len(home_to_purchase_list)

True

In [ ]:
week_over_week_data = pd.DataFrame({
    'Week': week1,
    'Home page to Product page conversion rate':home_to_product_list,
    'Product Page to Purchase page conversion rate': product_to_purchase_list,
    'Home page to Purchase page conversion rate': home_to_purchase_list
})

In [ ]:
week_over_week_data.head()

,Week,Home page to Product page conversion rate,Product Page to Purchase page conversion rate,Home page to Purchase page conversion rate
0,2023-w44,0.236080,0.012579,0.002970
1,2023-w45,0.269673,0.011475,0.003095
2,2023-w46,0.237876,0.014060,0.003344
3,2023-w47,0.251921,0.013559,0.003416
4,2023-w48,0.258298,0.011682,0.003018


creating a transpose here

In [ ]:
week_over_week_data.set_index('Week', inplace=True)

In [ ]:
temp_week_over_week_data = week_over_week_data.transpose()
temp_week_over_week_data.head()

Week,2023-w44,2023-w45,2023-w46,2023-w47,2023-w48
Home page to Product page conversion rate,0.236080,0.269673,0.237876,0.251921,0.258298
Product Page to Purchase page conversion rate,0.012579,0.011475,0.014060,0.013559,0.011682
Home page to Purchase page conversion rate,0.002970,0.003095,0.003344,0.003416,0.003018


In [ ]:
conversions = temp_week_over_week_data.index.tolist()

In [ ]:
conversions

['Home page to Product page conversion rate',
 'Product Page to Purchase page conversion rate',
 'Home page to Purchase page conversion rate']

In [ ]:
delta_week = pd.DataFrame()
for i in range(len(week_label)-1):
  x = temp_week_over_week_data[week_label[i+1]] - temp_week_over_week_data[week_label[i]]
  x = x.tolist()
  a = f'Week {week_label[i+1]} to Week {week_label[i]} Delta'
  delta_week.insert(i,a,x)

In [ ]:
delta_week.index = conversions

In [ ]:
 delta_week

,Week 2023-w45 to Week 2023-w44 Delta,Week 2023-w46 to Week 2023-w45 Delta,Week 2023-w47 to Week 2023-w46 Delta,Week 2023-w48 to Week 2023-w47 Delta
Home page to Product page conversion rate,0.033593,-0.031797,0.014045,0.006377
Product Page to Purchase page conversion rate,-0.001103,0.002584,-0.000500,-0.001877
Home page to Purchase page conversion rate,0.000125,0.000250,0.000071,-0.000398


Rule: Home page to purchase page conversion rate. Sorting the values and then based on that, returning the values from week_to_week_data

In [ ]:
sorted_df = delta_week.sort_values(by='Home page to Purchase page conversion rate', axis=1, ascending=False)

In [ ]:
sorted_df

,Week 2023-w46 to Week 2023-w45 Delta,Week 2023-w45 to Week 2023-w44 Delta,Week 2023-w47 to Week 2023-w46 Delta,Week 2023-w48 to Week 2023-w47 Delta
Home page to Product page conversion rate,-0.031797,0.033593,0.014045,0.006377
Product Page to Purchase page conversion rate,0.002584,-0.001103,-0.000500,-0.001877
Home page to Purchase page conversion rate,0.000250,0.000125,0.000071,-0.000398


Running Highest

In [ ]:
highest = sorted_df.columns[0]
lowest = sorted_df.columns[-1]

In [ ]:
h1 = highest.split(' ')[1]
h2 = highest.split(' ')[4]

In [ ]:
new_list_highest = [h1,h2]

In [ ]:
highest_home_to_purchase_page_conversion = week_over_week_data[week_over_week_data.index.isin(new_list_highest)]

In [ ]:
highest_home_to_purchase_page_conversion

,Home page to Product page conversion rate,Product Page to Purchase page conversion rate,Home page to Purchase page conversion rate
Week,,,
2023-w45,0.269673,0.011475,0.003095
2023-w46,0.237876,0.014060,0.003344


Running Lowest

In [ ]:
l1 = lowest.split(' ')[1]
l2 = lowest.split(' ')[4]

new_list_lowest = [l1,l2]
lowest_home_to_purchase_page_conversion = week_over_week_data[week_over_week_data.index.isin(new_list_lowest)]

In [ ]:
lowest_home_to_purchase_page_conversion

,Home page to Product page conversion rate,Product Page to Purchase page conversion rate,Home page to Purchase page conversion rate
Week,,,
2023-w47,0.251921,0.013559,0.003416
2023-w48,0.258298,0.011682,0.003018


In [ ]:
print(sorted_df)

                                               Week 2023-w46 to Week 2023-w45 Delta  \
Home page to Product page conversion rate                                 -0.031797   
Product Page to Purchase page conversion rate                              0.002584   
Home page to Purchase page conversion rate                                 0.000250   

                                               Week 2023-w45 to Week 2023-w44 Delta  \
Home page to Product page conversion rate                                  0.033593   
Product Page to Purchase page conversion rate                             -0.001103   
Home page to Purchase page conversion rate                                 0.000125   

                                               Week 2023-w47 to Week 2023-w46 Delta  \
Home page to Product page conversion rate                                  0.014045   
Product Page to Purchase page conversion rate                             -0.000500   
Home page to Purchase page conversion rat

In [ ]:
highest_lowest_data = pd.concat([highest_home_to_purchase_page_conversion,lowest_home_to_purchase_page_conversion])
highest_lowest_data

,Home page to Product page conversion rate,Product Page to Purchase page conversion rate,Home page to Purchase page conversion rate
Week,,,
2023-w45,0.269673,0.011475,0.003095
2023-w46,0.237876,0.014060,0.003344
2023-w47,0.251921,0.013559,0.003416
2023-w48,0.258298,0.011682,0.003018
